# Deep Learning Image Transfer (Convolutional Neural Network)

### Importing the libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.9.0'

## Data Preprocessing

### Preprocessing the Training set

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

training_set = train_datagen.flow_from_directory(
    'dataset/training_set',
    target_size=(64,64), # changed to 64 for quick process but highest value will help to get accurate result.
    batch_size=32,
    class_mode='binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_set = test_datagen.flow_from_directory(
    'dataset/test_set',
    target_size=(64,64), # changed to 64 for quick process but highest value will help to get accurate result.
    batch_size=32,
    class_mode='binary')

Found 2000 images belonging to 2 classes.


## Building the CNN

### Initialising the CNN

In [5]:
cnn = tf.keras.models.Sequential()

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-12-30 17:33:47.657860: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-30 17:33:47.658016: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


### Convolution

In [6]:
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu',input_shape=[64,64,3])) #3 for color(RGB)

### Pooling

In [7]:
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=2,strides=2))

### Adding a second convolutional layer

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu')) # input shape only pass at initial time.
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=2,strides=2))

### Flattening

In [9]:
cnn.add(tf.keras.layers.Flatten())

### Full Connection

In [10]:
cnn.add(tf.keras.layers.Dense(units=128,activation='relu'))

### Output Layer

In [11]:
cnn.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

## Training the CNN

### Compiling the CNN

In [12]:
cnn.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

### Model Summary

In [13]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)               8

### Training the CNN on the Training set and evaluating it on the Test set

In [14]:
cnn.fit(x = training_set, validation_data=test_set, epochs=25)

Epoch 1/25


2022-12-30 17:33:48.308326: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-30 17:33:48.518801: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


250/250 [==============================] - ETA: 0s - loss: 0.6916 - accuracy: 0.5485

2022-12-30 17:34:04.422058: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


250/250 [==============================] - 19s 72ms/step - loss: 0.6916 - accuracy: 0.5485 - val_loss: 0.6670 - val_accuracy: 0.6215
Epoch 2/25
250/250 [==============================] - 18s 72ms/step - loss: 0.6483 - accuracy: 0.6270 - val_loss: 0.6762 - val_accuracy: 0.5825
Epoch 3/25
250/250 [==============================] - 18s 71ms/step - loss: 0.6060 - accuracy: 0.6713 - val_loss: 0.6021 - val_accuracy: 0.6765
Epoch 4/25
250/250 [==============================] - 18s 70ms/step - loss: 0.5668 - accuracy: 0.7051 - val_loss: 0.5371 - val_accuracy: 0.7295
Epoch 5/25
250/250 [==============================] - 17s 68ms/step - loss: 0.5269 - accuracy: 0.7346 - val_loss: 0.5152 - val_accuracy: 0.7545
Epoch 6/25
250/250 [==============================] - 17s 68ms/step - loss: 0.5007 - accuracy: 0.7518 - val_loss: 0.5146 - val_accuracy: 0.7575
Epoch 7/25
250/250 [==============================] - 17s 69ms/step - loss: 0.4824 - accuracy: 0.7696 - val_loss: 0.4875 - val_accuracy: 0.7685
Epo

## Making a single prediction

In [20]:
import numpy as np
from keras.utils import load_img, img_to_array
test_image = load_img('dataset/sample_data/dog.jpeg',target_size = (64,64))
test_image = img_to_array(test_image)
test_image = np.expand_dims(test_image,axis=0)#expanding the array size (batching)
result = cnn.predict(test_image)
print(training_set.class_indices)

1/1 [==============================] - 0s 29ms/step
{'cats': 0, 'dogs': 1}


In [22]:
if result[0][0] == 1:
  print("dog")
else:
  print('cat')

dog
